In [ ]:
import pandas as pd
import numpy as np
import os
import re

# ============================================================
# 🧭 Y14_경찰청 지구대·파출소 수치화 전처리 (법정동코드 기반 시·구·동 파생)
# - 파일 로드
# - 법정동코드로 시·구·동 파생
# - 구·동별 지구대/파출소 수 집계 및 저장
# ============================================================

# ===== 0) 경로 =====
relative_data_path = '../../data/raw/'
file_name = 'Y14_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv'
input_file = os.path.join(relative_data_path, file_name)  # 변수 이름을 input_file로

output_dir = '../../data/preprocessing'
os.makedirs(output_dir, exist_ok=True)
output_file = os.path.join(
    output_dir,
    'Y14_전처리_지구대파출소_설치수_20241231기준.csv'
)

# ===== 1) 로드 =====
df = pd.read_csv(input_file, sep="\t", encoding="utf-8-sig")
df.columns = df.columns.str.strip()
df.info()

# ===== 2) 법정동코드로 시·구·동 파생 =====
df["법정동코드_10자리"] = (
    df["법정동코드"]
    .fillna(0)
    .astype(np.int64)
    .astype(str)
    .str.zfill(10)
)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2045 entries, 0 to 2044
Data columns (total 20 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   연번      2045 non-null   object 
 1   시도청     2031 non-null   object 
 2   경찰서     2031 non-null   object 
 3   관서명     2031 non-null   object 
 4   구분      2031 non-null   object 
 5   전화번호    2031 non-null   object 
 6   주소      2031 non-null   object 
 7   입력주소    2031 non-null   object 
 8   X       2031 non-null   float64
 9   Y       2031 non-null   float64
 10  CLSS    2031 non-null   object 
 11  PNU     2029 non-null   float64
 12  주소구분    2031 non-null   object 
 13  표준신주소   2017 non-null   object 
 14  표준구주소   2031 non-null   object 
 15  우편번호    1967 non-null   float64
 16  행정동코드   2004 non-null   float64
 17  행정동명    2004 non-null   object 
 18  법정동코드   1997 non-null   float64
 19  법정동명    2004 non-null   object 
dtypes: float64(6), object(14)
memory usage: 319.7+ KB


In [3]:
# ===== 2) 서울특별시 데이터만 필터링 =====
df_seoul = df[df['주소'].str.contains('서울특별시', na=False)].copy()
print(f"✅ [2단계] 서울특별시 데이터 필터링 완료: {df_seoul.shape[0]:,}행")

# ===== 3) 주소에서 '지역구' 추출 (서울만) =====
def extract_district(address):
    if pd.isna(address):
        return None
    match = re.search(r'([가-힣]+구)', str(address))  # 서울은 전부 '구' 단위
    return match.group(1) if match else None

df_seoul['지역구'] = df_seoul['주소'].apply(extract_district)

print("\n✅ [3단계] 주소에서 지역구 추출 완료")
print(df_seoul[['주소', '지역구']].head(10))
print(f"📊 서울특별시 지역구 결측치 수: {df_seoul['지역구'].isnull().sum()}")

# ===== 4) 지역구별 지구대/파출소 수 집계 및 수치화 =====
df_grouped = (
    df_seoul.groupby('지역구')
    .size()
    .reset_index(name='지역구별 지구대/파출소 수')
    .sort_values(by='지역구', ascending=True)
    .reset_index(drop=True)
)

# ===== 5) 결과 저장 =====
df_grouped.to_csv(output_file, index=False, encoding='utf-8-sig')
print(f"\n💾 [5단계] '{os.path.basename(output_file)}' 저장 완료")
print(f"📊 최종 행 수: {len(df_grouped):,} 행")
print("\n📌 저장 컬럼:", df_grouped.columns.tolist())
print(df_grouped.head(10))

✅ [2단계] 서울특별시 데이터 필터링 완료: 236행

✅ [3단계] 주소에서 지역구 추출 완료
                                 주소  지역구
0                서울특별시 중구 퇴계로49길 13   중구
1               서울특별시 중구  퇴계로 375-1   중구
2               서울특별시 중구  동호로 5길 15   중구
3   서울특별시 중구  다산로 248 (신당동, 신당파출소)    중구
4                 서울특별시 중구  동호로 261   중구
5                 서울특별시 중구  퇴계로 178   중구
6                  서울특별시 중구 충무로56-1   중구
7               서울특별시 종로구  종로17길 4   종로구
8    서울특별시 종로구 삼일대로15길 19 1층(임시파출소)  종로구
9              서울특별시 종로구  세검정로 226   종로구
📊 서울특별시 지역구 결측치 수: 0

💾 [5단계] 'Y14_전처리_경찰청_전국 지구대 파출소 주소 현황_20241231_TM.csv' 저장 완료
📊 최종 행 수: 25 행

📌 저장 컬럼: ['지역구', '지역구별 지구대/파출소 수']
   지역구  지역구별 지구대/파출소 수
0  강남구              14
1  강동구               9
2  강북구               9
3  강서구              11
4  관악구               9
5  광진구              10
6  구로구               8
7  금천구               5
8  노원구               8
9  도봉구               8
